In [10]:
import os
import glob
import gzip

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm

import shapely.wkt as wkt
from shapely.geometry import Point, LineString, box
from shapely.ops import nearest_points

import lxml.etree as ET
import pickle

# Define the path to the folder containing the network directories
base_path = '../../../data/pop_1pm_policies_new/output_policies/output_networks/'
network_folders = glob.glob(os.path.join(base_path, 'network_d_*'))
districts = gpd.read_file("../../../data/visualisation/districts_paris.geojson")
base_output_links_no_policies = '../../../data/pop_1pm/output_1pm_cap_0.005/'

# Abstract

To do: die relative Analyse pro District machen. Wie hat sich capacity, freeflow speed und car volume in dem District geändert, in dem Policy eingeführt wurde? Und wie hat es sich netzwerkweit geändert?

In [11]:
def create_policy_key(folder_name):
    # Extract the relevant part of the folder name
    parts = folder_name.split('_')[1:]  # Ignore the first part ('network')
    district_info = '_'.join(parts).replace('d_', '')
    districts = district_info.split('_')
    return f"policy introduced in Arrondissement(s) {', '.join(districts)}"

# Function to read and convert CSV.GZ to GeoDataFrame
def read_network_data(folder):
    file_path = os.path.join(folder, 'output_links.csv.gz')
    if os.path.exists(file_path):
        # Read the CSV file with the correct delimiter
        df = pd.read_csv(file_path, delimiter=';')
        
        # Convert the 'geometry' column to actual geometrical data
        df['geometry'] = df['geometry'].apply(wkt.loads)
        
        # Create a GeoDataFrame
        gdf = gpd.GeoDataFrame(df, geometry='geometry')
        return gdf
    else:
        return None
    
# Plotting function
def plot_policy_data(key, df):
    arrondissement_number = key.replace('policy_', '').replace('_', ' ')
    arrondissement_number_cleaned = arrondissement_number.replace(" ", "_")

    print(f"Policy: {key}")
    
    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:2154")
    gdf = gdf.to_crs(epsg=4326)

    x_min = gdf.total_bounds[0] + 0.05
    y_min = gdf.total_bounds[1] + 0.05
    x_max = gdf.total_bounds[2]
    y_max = gdf.total_bounds[3]
    bbox = box(x_min, y_min, x_max, y_max)
    
    # Filter the network to include only the data within the bounding box
    gdf = gdf[gdf.intersects(bbox)]
    
    # Set up the plot
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    gdf[gdf['vol_car'] == 0].plot(ax=ax, color='lightgrey', linewidth=0.2, label='Network', zorder = 1)

    # Plot links with activity_count using the Viridis color scheme and log scale
    gdf.plot(column='vol_car', cmap='coolwarm', linewidth=1.5, ax=ax, legend=True,
             norm=LogNorm(vmin=gdf['vol_car'].min() + 1, vmax=gdf['vol_car'].max()),
             legend_kwds={'label': "Car volume", 'orientation': "vertical", 'shrink': 0.5})
    
    # districts.plot(ax=ax, facecolor='None', edgecolor='black', linewidth=1, label = "Arrondissements", zorder=3)

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    
    # Customize the plot
    plt.title(f"{arrondissement_number}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.legend()
    plt.savefig("results/network_policies_new/" + f"{arrondissement_number_cleaned}.png", dpi=300, bbox_inches='tight')
    # plt.show()
    
# Function to iterate over result_dic and perform the required operations
def analyze_geodataframes(result_dic):
    # Extract the base network data for comparison
    base_gdf = result_dic.get("base_network_no_policies")
    if base_gdf is not None:
        base_vol_car = base_gdf['vol_car'].sum()
        base_gdf_car = base_gdf[base_gdf['modes'].str.contains('car')]
        base_capacity_car = base_gdf_car['capacity'].sum() * 0.005
        base_freespeed_car = base_gdf_car['freespeed'].sum()

    for policy, gdf in result_dic.items():
        print(f"Policy: {policy}")
        
        # Filter edges where 'modes' contains 'car'
        gdf_car = gdf[gdf['modes'].str.contains('car')]
        total_capacity_car = round(gdf_car['capacity'].sum() * 0.005)
        print(f"Total capacity of edges with 'car' mode: {total_capacity_car}")
        
        total_freespeed_car = round(gdf_car['freespeed'].sum())
        print(f"Total freespeed of edges with 'car' mode: {total_freespeed_car}")

        total_vol_car = gdf['vol_car'].sum()
        print(f"Total 'vol_car': {total_vol_car}")

        if policy != "base_network_no_policies" and base_gdf is not None:
            vol_car_increase = ((total_vol_car - base_vol_car) / base_vol_car) * 100
            capacity_car_increase = ((total_capacity_car - base_capacity_car) / base_capacity_car) * 100
            freespeed_car_increase = ((total_freespeed_car - base_freespeed_car) / base_freespeed_car) * 100

            print(f"Percentage increase in 'vol_car': {vol_car_increase:.2f}%")
            print(f"Percentage increase in capacity (car edges): {capacity_car_increase:.2f}%")
            print(f"Percentage increase in freespeed (car edges): {freespeed_car_increase:.2f}%")
        
        print()

In [12]:
# Read all network data into a dictionary of GeoDataFrames
result_dic = {}
for folder in network_folders:
    policy_key = create_policy_key(os.path.basename(folder))
    gdf = read_network_data(folder)
    if gdf is not None:
        result_dic[policy_key] = gdf
        
base_network_no_policies = read_network_data(base_output_links_no_policies)
result_dic["base_network_no_policies"] = base_network_no_policies

In [13]:
for key, df in result_dic.items():
    print(key)
    # plot_policy_data(key, df)

policy introduced in Arrondissement(s) 5, 6
policy introduced in Arrondissement(s) 1, 2, 3
policy introduced in Arrondissement(s) 3, 4
policy introduced in Arrondissement(s) 16, 17, 18
policy introduced in Arrondissement(s) 13, 14, 15
policy introduced in Arrondissement(s) 8, 9, 10, 11
policy introduced in Arrondissement(s) 12, 13
policy introduced in Arrondissement(s) 2, 3, 4, 5, 6, 7
policy introduced in Arrondissement(s) 9, 10, 11
policy introduced in Arrondissement(s) 15, 16, 17, 18, 19
policy introduced in Arrondissement(s) 7, 8, 9, 10, 11, 12
policy introduced in Arrondissement(s) 13, 14
policy introduced in Arrondissement(s) 2, 3, 4, 5, 6
policy introduced in Arrondissement(s) 5
policy introduced in Arrondissement(s) 2
policy introduced in Arrondissement(s) 11, 12, 13, 14
policy introduced in Arrondissement(s) 12, 13, 14, 15
policy introduced in Arrondissement(s) 14, 15, 16, 17, 18
policy introduced in Arrondissement(s) 3, 4, 5, 6, 7
policy introduced in Arrondissement(s) 17, 18

In [14]:
analyze_geodataframes(result_dic)

Policy: policy introduced in Arrondissement(s) 5, 6
Total capacity of edges with 'car' mode: 154159
Total freespeed of edges with 'car' mode: 253044
Total 'vol_car': 175287
Percentage increase in 'vol_car': -7.73%
Percentage increase in capacity (car edges): -3.55%
Percentage increase in freespeed (car edges): -0.00%

Policy: policy introduced in Arrondissement(s) 1, 2, 3
Total capacity of edges with 'car' mode: 155468
Total freespeed of edges with 'car' mode: 253044
Total 'vol_car': 173564
Percentage increase in 'vol_car': -8.63%
Percentage increase in capacity (car edges): -2.73%
Percentage increase in freespeed (car edges): -0.00%

Policy: policy introduced in Arrondissement(s) 3, 4
Total capacity of edges with 'car' mode: 156174
Total freespeed of edges with 'car' mode: 253022
Total 'vol_car': 180906
Percentage increase in 'vol_car': -4.77%
Percentage increase in capacity (car edges): -2.29%
Percentage increase in freespeed (car edges): -0.01%

Policy: policy introduced in Arrondis

## Save the results_dic, so that we can use it for ML methods

In [15]:
# Save the dictionary
with open('results/result_dic.pkl', 'wb') as f:
    pickle.dump(result_dic, f)